# Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.

###* Create a Jupyter Notebook file called `mission_to_mars.ipynb` and use this to complete all of your scraping and analysis tasks. The following outlines what you need to scrape.

In [1]:
# Imports and dependencies
from bs4 import BeautifulSoup as bs
import requests
import os
from splinter import Browser
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd


###  Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [2]:
# set url's
#### url to mars mission news
url_news="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
##### URL to mars facts
url_facts='https://space-facts.com/mars/'
#### Url to astrogeology
hemispheres_home_url = 'https://astrogeology.usgs.gov'
url_hemispheres = hemispheres_home_url + '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'


In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\luisf\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [4]:
browser.visit(url_news)


In [5]:
html = browser.html
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(html, 'lxml')
#print(soup.prettify())

In [7]:
#just the newest news
slide=soup.find("li",class_="slide")
news_title = slide.h3.text
news_p = slide.find('div', class_='rollover_description_inner').text.strip()
print('---------------')
print(news_title)
print('---------------')
print(news_p)

AttributeError: 'NoneType' object has no attribute 'h3'

# No to do it told by instructor.

##### JPL Mars Space Images - Featured Image
* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).
* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.
* Make sure to find the image url to the full size `.jpg` image.
* Make sure to save a complete url string for this image.




### Mars Facts

* Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [ ]:
#read the data with Pandas
table = pd.read_html(url_facts)
table

In [ ]:
# Get the data to a DataFrame
facts_df = table[0]
#Rename Columns
facts_df.columns=['Description','Value']
# Remove colon
facts_df["Description"] = facts_df["Description"].replace({':':''}, regex=True)

facts_df

In [ ]:
#convert table to HTML code
html_table = facts_df.to_html()
html_table

In [ ]:
#replace new line with white space
fact_table = html_table.replace(r'\\n',' ')
#, regex=True) 
#replace('\n', '')#, regex=True) 
#(r'\\n',' ', regex=True) 
fact_table

In [ ]:
#save html in seprate file 
facts_df.to_html('mars_facts.html')

### Mars Hemispheres

* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [ ]:
#### Url to astrogeology
hemispheres_home_url = 'https://astrogeology.usgs.gov'
url_hemispheres = hemispheres_home_url + '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [ ]:
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

In [ ]:
# create beautifulsoup object
html = browser.html
soup_hemispheres = bs(html, 'lxml')

In [ ]:
links = browser.find_by_css("a.product-item h3")
len(links)

In [ ]:
hemisphere_imgage_urls = []

In [ ]:
# # Finding the hemispheres data through their HTML divisions
# mars_hemispheres = soup_hemispheres.find('div', class_='collapsible results')
# hemispheres = mars_hemispheres.find_all('div', class_='item')

# for i in range(3):
#     hemispheres = {}
    
# #     hemisphere = hemi.find('div', class_="description")
# #     title = hemisphere.h3.text
# #     title = title.strip('Enhanced')
    
#     browser.find_by_css("a.product-item h3")[i].click()
    
#     sample_elem = browser.links.find_by_text('Sample').first
#     hemispheres['img_url'] = sample_elem['href']
    
#     hemispheres['title'] = browser.find_by_css('h2.title').text
    
#     hemisphere_imgage_urls.append(hemispheres)
    
#     browser.back()
    
# browser.quit()
    
    
# print(hemisphere_imgage_urls)

# #<a href="/search/map/Mars/Viking/cerberus_enhanced" 
# #class="itemLink product-item"><h3>Cerberus Hemisphere Enhanced</h3></a>

In [ ]:

    # Finding the hemispheres data through their HTML divisions
mars_hemispheres = soup_hemispheres.find('div', class_='collapsible results')
hemispheres = mars_hemispheres.find_all('div', class_='item')

    # Empty list for hemispheres' Title and image_urls
mars_images = []

    # Iterate through each hemisphere data
for hemi in hemispheres:
        # Get Titles
    hemisphere = hemi.find('div', class_="description")
    title = hemisphere.h3.text
    title = title.strip('Enhanced')

        # Get Images
    end_link = hemisphere.a["href"]
    browser.visit(hemispheres_home_url + end_link)

    image_html = browser.html
    image_soup = BeautifulSoup(image_html, 'html.parser')

    image_link = image_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']

        # Storage Dictionary
    image_dict = {}
    image_dict['Title'] = title
    image_dict['ImageURL'] = image_url

        # Add data to empty list "mars_images"
    mars_images.append(image_dict)

# Close browser window
browser.quit()
    
mars_images

In [ ]:
# Create dictionary for all info scraped from sources above
mars_dict={
    "news_title":news_title,
    'news_p':news_p,
    "fact_table":fact_table,
    "mars_images":hemisphere_imgage_urls
}
mars_dict